# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **religious events or traditions** (see below) for the region of **Burbank, California, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Burbank, California, United States** to Ann Arbor, USA. In that case at least one source file must be about **Burbank, California, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Burbank, California, United States** and **religious events or traditions**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **religious events or traditions**?  For this category you might consider calendar events, demographic data about religion in the region and neighboring regions, participation in religious events, or how religious events relate to political events, social movements, or historical events.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

# Solution

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data

from: http://www.thearda.com/Archive/Files/Descriptions/NES16.asp

The data were downloaded from the Association of Religion Data Archives, www.TheARDA.com, and were collected by 

## Files

 * `ARDA_2016questions.txt`: The list of questions asked, which correspond to the data file. I have marked these with the questions I found interesting
 * `ARDA_2016survey.csv`: The raw files containing survey responses
 

In [82]:
# some constants

# The code for the repsondant ID
identity_code = 'V160001'


### Filter out relevant questions

This is a map from the question code which maps the column to the question text, and other info about the question

In [75]:
class Question:
    """Class for storing information about each question, question text, meaning of answers, etc."""
    def __init__(self, code):
        self.code = code.strip()
        self.answers = {}
    
    def __str__(self):
        return "{}: {} answers, question = {}"\
            .format(self.code, len(self.answers), self.text)
    
    def __repr__(self):
        return self.__str__()
    
    def set_text(self, text):
        self.text = text
        
    def add_answer(self, code, val):
        self.answers[int(code)] = val
        

### Read question file

In [92]:
questions = {}
with open('a4_project_data/ARDA_2016questions.txt') as f:
    include, readnext = False, 0
    for lin in f:
        lin = lin.strip()
        
        if include:
            if readnext == 4:
                ques.set_text(lin)
            elif readnext == 0:
                try:
                    # split answer code from description
                    code, rest = lin.split(')', 1)
                    # take all tokens but last two (count and percentage)
                    ans = rest.strip().split()[:-2]
                    ques.add_answer(code, ' '.join(ans))
                except ValueError:
                    # hit an empty line after reading answers
                    # print(lin)
                    include = False
                
        if (lin.startswith('*')):
            code = lin[lin.index(')') + 2:]
            ques = Question(code)
            questions[code] = ques
            include = True
            readnext = 5
            
        if readnext > 0:
            readnext -= 1
            
# questions['V162342']
# help(str.split)

In [102]:
# questions related to religion
religionqs = tuple(filter(lambda c: 'religi' in questions[c].text.lower(), questions.keys()))
rq2 = list(questions.keys())
rq2 = rq2[rq2.index(religionqs[0]):rq2.index(religionqs[-1])]
religionqs = rq2

### Read survey file, filter all but relevant questions

In [12]:
df0 = pd.read_csv('a4_project_data/ARDA_2016survey.csv')

#### Filter out questions in the question file not found in the data

In [95]:
fnd, nf = [identity_code],0
for k, q in questions.items():
    try:
        df0[k]
        fnd.append(k)
    except KeyError:
        nf += 1
        print('question not found in file:', q)
        
df1 = df0[fnd].rename(columns={identity_code: 'Participant'}).set_index('Participant')
df1.columns.name = 'Question Code'
df1.head()

question not found in file: V162342: 11 answers, question = Please mark how well the following pair of words describes you, even if one word describes you better than the other. 'Conventional, uncreative' describes me...


Question Code,V161071,V161074,V161077,V161080,V161081,V161085,V161086,V161116,V161118,V161120,...,V162099,V162101,V162105,V162126,V162127,V162160,V162168,V162170,V162255,V162255A
Participant,,,,,,,,,,,,,,,,,,,,,
300001,1,1,2,2,2,2,0,5,4,5,...,70,70,50,1,5,1,2,1,2,5
300002,1,1,1,2,2,2,0,3,3,3,...,50,60,50,1,5,5,1,3,1,4
300003,2,2,2,2,2,2,50,2,2,2,...,60,50,50,1,4,4,2,1,1,3
300004,1,1,1,1,2,2,0,3,3,3,...,50,50,50,1,3,3,4,2,2,5
300006,1,2,1,2,2,2,15,2,2,4,...,85,70,70,1,3,3,4,2,1,4


In [104]:
[questions[k] for k in religionqs]



[V161241: 4 answers, question = Now on another topic.... do you consider religion to be an important part of your life, or not?,
 V161242: 6 answers, question = IF R SAYS THAT RELIGION IS IMPORTANT: Would you say your religion provides [some guidance in your day-to-day living, quite a bit of guidance, or a great deal of guidance / a great deal of guidance in your day-to-day living, quite a bit of guidance, or some guidance]?,
 V161243: 6 answers, question = Which of these statements comes closest to describing your feelings about the Bible?,
 V161244: 4 answers, question = Lots of things come up that keep people from attending religious services even if they want to. Thinking about your life these days, do you ever attend religious services, apart from occasional weddings, baptisms or funerals?,
 V161245: 7 answers, question = IF R ATTENDS RELIGIOUS SERVICES: Do you go to religious services [every week, almost every week, once or twice a month, a few times a year, or never / never, a f

### Next task: identify the participant's religion using questions V161247A, V161247B, V161248